### Analyzing MEPS-HC Data with SAS® 9.4M6 
#### By Pradip K. Muhuri

#####  Generate output on the number of observations and variables

In [ ]:
options nocenter nonumber nodate;
options formchar="|----|+|---+=|-/\<>*";
proc sql;
select nobs format=comma7.
       ,nvar 
    from dictionary.tables
    where libname = 'SASHELP' and memname = 'CLASS';
 quit;


In [ ]:
options nocenter nonumber nodate nonotes nosource;
ods html close;
options formchar="|----|+|---+=|-/\<>*";
* Create macro variables;
proc sql noprint;
select nobs, nvar 
     into :nobs_m, :nvar_m
    from dictionary.tables
    where libname = 'SASHELP' and memname = 'HEART';
 quit;
 %put Number of observations: %left(%qsysfunc(putn(&nobs_m, comma7.)));
 %put Number of variables: %left(%qsysfunc(putn(&nvar_m, comma7.)));


In [ ]:
options nonotes nosource nodate nonumber;
ods html close;
data _NULL_;
set SASHELP.HEART;
array char[*} $ _CHAR_;
array num[*} _NUMERIC_;
nCharVar  = dim(char);
nNumerVar = dim(num);
put "SASHELP HEART: " nCharVar= nNumerVar= ;
stop;   /* stop processing after first observation */
run;

In [ ]:
%showlog

In [ ]:
* Generate a list of all variables and their attributes in the data set;
options nocenter nonumber nodate;
options formchar="|----|+|---+=|-/\<>*";
proc contents data=sashelp.class varnum;
ods select position;
run;


In [ ]:
* Generate a list of all variables and their attributes in the data set;
options nocenter nonumber nodate;
options formchar="|----|+|---+=|-/\<>*";
proc contents data=sashelp.class p;
ods select variables;
run;


In [ ]:
options nocenter nonumber nodate;
options formchar="|----|+|---+=|-/\<>*";
data mytable (keep=name type length label);
   set sashelp.vcolumn;
   where libname='SASHELP' and memname='CLASS';
run;
proc print data=mytable noobs;
run;

In [ ]:
options nocenter nonumber nodate;
options formchar="|----|+|---+=|-/\<>*";
proc sql outobs=10;
     select name, type, length, label from sashelp.vcolumn
      where libname='SASHELP' and memname='HEART';
quit;

In [ ]:
* Create a macro variable that holds a horizontal list of variables of interest;
options nocenter nonumber nodate nosource;
proc sql noprint;
select name
  into :varlist separated by ' ' 
 from dictionary.columns
 where libname= "PUFMEPS" and memname = "H201"
 and (name like "INSCOP%" or name like "KEY%" or name like "PERWT%");
quit;
%put &=varlist;

In [ ]:
options nocenter nonumber nodate nosource;
libname pufmeps  'C:\Data' access=readonly;
* Print the meta data for the variables of interest;
proc sql ;
select name, type, label
  from dictionary.columns
 where libname= "PUFMEPS" and memname = "H201"
 and (name like "INSCOP%" or name like "KEY%" or name like "PERWT%");
quit;


In [ ]:
* Generate a listing of MEPS data files dynamically;
ods excel file = 'C:\Data\List_MEPS_Datasets.xlsx'
   options (embedded_titles='on'  sheet_name='List'); 
title "Listing of MEPS Data Files (SAS Data Sets) from  C:\Data";
libname pufmeps  'C:\Data' access=readonly;
proc sql number;
  select memname 
         ,nobs format=comma10.
         ,nvar format=comma7.
         ,DATEPART(crdate) format date9. as Date_created label='Creation Date'
         ,TIMEPART(crdate) format timeampm. as Time_created label='Creation Time'
     from dictionary.tables
       where libname = 'PUFMEPS' 
        and memtype = 'DATA'
   order by Date_created desc;
  quit;
ods excel close;
ods listing;